# Climbing database project

### Project goal 
Extract usable data from 15 years of climbing logs and do some basic analysis.

### Learning goals
- Getting a table out of a Google sheet.
- Data model design and intro to using `dataclasses` to implement the data model.
  - See https://www.youtube.com/watch?v=vRVVyl9uaZc for a nice intro.
- Data persistence and benefits/limitations of the Python `shelve` library package.
- Efficiently extracting information from natural-language text without using anything fancy.
- Type annotations in Python and how they help you write better code more quickly (esp. paired with Copilot).
  - See https://www.youtube.com/watch?v=dgBCEB2jVU0&t=189s for a nice intro.
- Making a fancy histogram plot with matplotlib.

## Raw data: Google sheets

Our climbing database is a google spreadsheet with one sheet (table) per year going back
to 2009.

Each sheet has one row per climbing day with columns Date, Place/Climbers, and Log.
For historical reasons these are called "Date", "Climb", and "Comments". Example:
```
Date: 2024-04-26
Climb: TA AS Rumney (Tom and Aneta climbed at Rumney)
Comments: Armed, Obi (AS), Jedi Mind Tricks (TA working moves),
  Centerpiece (AS 2xlead 2h, TA redpoint), Social O (TA,  AS to the top with 2 hangs)
```
The comments include information on climbs but are often free-form. Ideally they are
comma-separated lists of climbs with informative comments. In practice the comments
are full of typos, inconsistencies and shorthand.

`Armed` is short for `Armed, Dangerous, and Off My Medication`. 
`AS 2xlead 2h, TA redpoint` means that Aneta lead the climb twice but had to hang on the
rope both times. Tom did that climb cleanly without hanging on the rope.

The comments can be even harder to parse (especially for older entries):
```
Nice day but HOT in the end (in the sun).  TA feeling stronger, two tries on
Orangahang (still close, felt really good on second run but got stuck making last
clip).  AS did Masterpiece 10a and Tropicana (in blazing sun).
```

### Reading a table from Google sheets

If you open a Google sheets document, you will see a URL like:
```
https://docs.google.com/spreadsheets/d/1gnVwOr_t_9ABWPKxK0DyoRhoksbb2ttaWf6u62aWeo4/edit#gid=0
```
- The long string after the `spreadsheets/d` is the `doc_id`.
- The `gid=...` is the `gid`

The code below will read this into an astropy `Table`.

#### Caveat

This requires setting the sharing of the document to *Anyone on the internet with the
link can view*. In general this is safe enough because the `doc_id` is not guessable,
but if your code is shared publicly then you should ensure the `doc_id` stays private
(if the data are sensitive in any way).

In this project there is a module `doc_ids.py` that is only local and not in the 
GitHub repo.


In [1]:
import functools
import requests
from astropy.table import Table

# See https://stackoverflow.com/questions/33713084 (2nd answer)
GOOGLE_SHEET_URL = (
    "https://docs.google.com/spreadsheets/d/{doc_id}/export?"
    "format=csv"
    "&id={doc_id}"
    "&gid={gid}"
)


@functools.lru_cache(maxsize=1)
def read_google_sheet(doc_id: str, gid: str) -> Table:
    url = GOOGLE_SHEET_URL.format(doc_id=doc_id, gid=gid)
    req = requests.get(url, timeout=30)
    if req.status_code != 200:
        raise ValueError(f"Failed to get google sheet: {req.status_code}")

    out = Table.read(req.text, format="csv", fill_values=[])
    return out


doc_id = "1gnVwOr_t_9ABWPKxK0DyoRhoksbb2ttaWf6u62aWeo4"
gid = "0"
table = read_google_sheet(doc_id, gid)
table[-5:]

Date,Climb,Comments
str10,str25,str52
2024-04-20,TA AS Rumney,"Armed, Obi (AS), Jedi Mind Tricks (TA working moves)"
2024-04-26,"TA Rumney w/ Nancy, Keith","Metamorphasis, Sesame, Armed"
2024-04-28,AS TakTak,"Endurance: V2, V2, V1 boulders in the cave"


## Data model overview

The first part of any project like this is deciding on a data model for representing the
data.

### Route information
The data model starts with a database of route information: list of route names and grade.
The route information is independent of our climbing on them.

### Our climbs on the routes
This is hierarchical:

List of climbing days (`ClimbingDay`): we went climbing on a certain day.

`ClimbingDay`: A day of climbing from the log (i.e. one row in the sheet) with the 
`date`, `place/climbers`, and `comments` from the sheet. In addtion this a list of 
`ClimbEntry` objects that get parsed from the `comments`.

`ClimbEntry`: One particular route in `comments` where there may be multiple ascents (or
attempts) of the route by one or both of Tom/Aneta. E.g. `Centerpiece (AS 2xlead 2h, TA
redpoint)`. This data class includes route information and a list of `ClimbEvent`.

`ClimbEvent`: One person doing one ascent or attempt of a route.

### Implementation: dataclasses

This is done in `core.py` -- let's take a look!

## Data persistence

A database provides a method for `data persistence`, i.e. ensuring that data which gets
entered into a program (and is in memory) is available at a later time.

There are many ways in Python to implement data persistence. For this project I choose
a simple and lazy way using the `shelve` module. I don't recommend this for production
projects but in this case it made the coding simple and fast.

In [2]:
import shelve
from pprint import pprint

# We open the shelve with a context manager to ensure it is closed properly
with shelve.open("climb_info") as db:
    print(f"Total of {len(db)} climbs in the shelve")
    print(list(db.keys())[:5])
    pprint(db["Centerpiece"])

Total of 91 climbs in the shelve
['Big Easy', 'Holderness Arete', 'Holderness Corner', 'Idiot Deluxe', 'Med Dose Madness']
ClimbInfo(name='Centerpiece', grade='10d', aliases=['Centerpiece'])


In [3]:
with shelve.open("climbing_days") as db:
    print(f"Total of {len(db)} climbing days in the shelve")
    print(list(db.keys())[:5])
    pprint(db["2024-04-26"], width=200)

Total of 181 climbing days in the shelve
['2021-10-01', '2021-10-02', '2021-11-11', '2022-10-21', '2022-10-22']
ClimbingDay(date='2024-04-26',
            log_text='Armed, Obi (AS), Jedi Mind Tricks (TA working moves), Centerpiece (AS 2xlead 2h, TA redpoint), Social O (TA,  AS to the top with 2 hangs)',
            place_and_climbers='TA AS Rumney',
            climb_entries=[ClimbEntry(name_approx='Armed',
                                      comment='',
                                      climbers=['TA', 'AS'],
                                      climb_info=ClimbInfo(name='Armed & Dangerous',
                                                           grade='10b',
                                                           aliases=['Armed & Dangerous', 'Arm & Dangerous', 'Armed and Dangerous', 'Arm and Dangerous', 'Armed']),
                                      climb_events=[ClimbEvent(climber='TA', hang=False), ClimbEvent(climber='AS', hang=False)],
                             

## Data entry

A key challenge here is extracting correct climbing entries, routes, and climb events 
from the free-format and often incomplete comments. There are a lot and this quickly
gets tedious.

My strategy was to make a custom data entry tool which does a few things:
- Make a best effort at parsing each sheet comment and creating an initial `ClimbEntry`
- Rely on the user to validate or correct the information.
  - In practice about half the time the initial guess is OK
- Mostly use single keypresses to speed the process. After some practice this is fast.

In the end it takes about an hour to enter of year of data.

## Data analysis

In the end we want to gain insight from the database! I've made a notebook that shows
using the database.